# tAIrritory

#### tAIrritory is a 7x3 turn-based strategy game where two players (P1 and P2) compete to dominate the middle row. Each player controls 6 pieces (3 Type-A and 3 Type-B), with unique movement and interaction rules. The game challenges players to outmaneuver their opponent through tactical decisions.
#### Key Features
#### Victory Condition: Control the middle row (Row 4) when the game ends, or face a draw.
#### Dynamic Rules: Pieces can block or interchange with opponents, and players must always make a valid move.
#### AI Integration: Train a reinforcement learning agent (DQN/PPO) to compete with human players.
#### GUI Frontend: Play the game through an intuitive, interactive interface.
#### tAIrritory combines strategy and artificial intelligence, offering fun gameplay and a platform to explore AI learning.

### Import neccessary packages

In [2]:
import numpy as np
import gymnasium as gym
import pygame
import torch

#### Create the GYMnasium environment and implement reset() and step() functions

In [3]:
class TAIrritoryEnv(gym.Env):

    metadata = {"render.modes": ["human"]}

    def __init__(self):
        super(TAIrritoryEnv, self).__init__()
        self.board = np.zeros((7, 3), dtype=int)
        self._initialize_board()
        self.action_space = gym.spaces.Box(low=0, high=np.array([6, 2, 6, 2]), shape=(4,), dtype=int)
        self.observation_space = gym.spaces.Box(low=-2, high=2, shape=(7, 3), dtype=np.int8)
        self.current_player = 1
        self.done = False
        self.winner = None
    
    def _initialize_board(self):
        self.board[0] = [2, 1, 2]
        self.board[1] = [2, 1, 2]
        self.board[5] = [-2, -1, -2]
        self.board[6] = [-2, -1, -2]

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.board = np.zeros((7, 3), dtype=int)
        self._initialize_board()
        self.current_player = 1
        self.done, self.winner = False, None
        return self.board, {}
    
    def possible_move_for_piece(self, row, col) -> list:
        piece = self.board[(row, col)]
        possible_moves = []
        if self.current_player == 1:
            if row > 3:
                if piece == -1: 
                    if col-1 >= 0 and self.board[(row - 1, col-1)] in [0, 2]:
                        possible_moves.append([row - 1, col-1])
                
                    elif self.board[(row - 1, col)] in [0, 2]:
                        possible_moves.append([row - 1, col])
                
                    elif col+1 < 2 and self.board[(row - 1, col + 1)] in [0, 2]:
                        possible_moves.append([row - 1, col + 1])
                
                elif piece == -2: 
                    if col-1 >= 0 and self.board[(row - 1, col-1)] in [0, 1]:
                        possible_moves.append([row - 1, col-1])
                
                    elif self.board[(row - 1, col)] in [0, 1]:
                        possible_moves.append([row - 1, col])
                    
                    elif col+1 < 2 and self.board[(row - 1, col+1)] in [0, 1]:
                        possible_moves.append([row - 1, col+1])
        
        elif self.current_player == 2:
            if row < 3:
                if piece == 1:
                    if col-1 >= 0 and self.board[(row + 1, col-1)] in [-2, 0]:
                        possible_moves.append([row + 1, col-1])
                
                    elif self.board[(row + 1, col)] in [-2, 0]:
                        possible_moves.append([row + 1, col])
                    
                    elif col+1 < 2 and self.board[(row + 1, col+1)] in [-2, 0]:
                        possible_moves.append([row + 1, col+1])
                
                elif piece == 2:
                    if col-1 >= 0 and self.board[(row + 1, col-1)] in [-1, 0]:
                        possible_moves.append([row + 1, col-1])
                
                    elif self.board[(row + 1, col)] in [-1, 0]:
                        possible_moves.append([row + 1, col])
                    
                    elif col+1 < 2 and self.board[(row + 1, col+1)] in [-1, 0]:
                        possible_moves.append([row + 1, col+1])
        
        return possible_moves
    
    def _is_game_over(self, p_type):
        for row in range(7):
            for col in range(3):
                if (p_type == 1 and (self.board[row, col] == -1 or self.board[row, col] == -2)) or \
                (p_type == 2 and (self.board[row, col] == 1 or self.board[row, col] == 2)):
                    if len(self.possible_move_for_piece(row, col)) > 0:
                        return False
        return True

    def _calculate_reward(self):
        p1_count = np.sum((self.board[3] == -1) | (self.board[3] == -2))
        p2_count = np.sum((self.board[3] == 1) | (self.board[3] == 2))
        return 0 if p1_count == p2_count else 1 if p1_count < p2_count else -1
    
    def step(self, action):
        self.board[(action[0], action[1])], self.board[(action[2], action[3])] = \
        self.board[(action[2], action[3])], self.board[(action[0], action[1])]
        self.current_player = 1 if self.current_player == 2 else 2
        terminated = self._is_game_over(self.current_player)
        reward = self._calculate_reward() if terminated else 0
        truncated = False
        info = {}
        return self.board, reward, terminated, truncated, info
    
    def render(self, mode=None):
        if mode == "human":
            print("\n".join(" ".join(str(cell) for cell in row) for row in self.board))

    def close(self):
        pass
    


#### Register the new env to call it using gym.make()

In [4]:
gym.register(id="gymnasium_env/tAIrritory-v0", entry_point=TAIrritoryEnv)

#### Do mockup test

In [7]:
env = gym.make("gymnasium_env/tAIrritory-v0")
obs = env.reset()

while True:
    print(f"\nCurrent player: {env.unwrapped.current_player}")
    print(f"Board state\n{env.unwrapped.board}")
    
    action_input = input("Enter 4-digit action (from_row, from_col, to_row, to_col) or 'q' to quit: ")
    
    if action_input.lower() == 'q':
        break
    
    try:
        action = [int(digit) for digit in action_input]
        
        # Validate action
        if len(action) != 4:
            print("Invalid action. Must be 4 digits.")
            continue
        
        obs, reward, terminated, truncated, info = env.step(action)
        print(f"Action taken: {action}")
        print(f"Reward: {reward}")
        
        if terminated or truncated:
            print("Game over!")
            break
    
    except ValueError:
        print("Invalid input. Please enter 4 digits.")

env.close()


Current player: 1
Board state
[[ 2  1  2]
 [ 2  1  2]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [-2 -1 -2]
 [-2 -1 -2]]
Action taken: [5, 0, 4, 0]
Reward: 0

Current player: 2
Board state
[[ 2  1  2]
 [ 2  1  2]
 [ 0  0  0]
 [ 0  0  0]
 [-2  0  0]
 [ 0 -1 -2]
 [-2 -1 -2]]
Action taken: [1, 0, 2, 0]
Reward: 0

Current player: 1
Board state
[[ 2  1  2]
 [ 0  1  2]
 [ 2  0  0]
 [ 0  0  0]
 [-2  0  0]
 [ 0 -1 -2]
 [-2 -1 -2]]
Action taken: [4, 0, 3, 0]
Reward: 0

Current player: 2
Board state
[[ 2  1  2]
 [ 0  1  2]
 [ 2  0  0]
 [-2  0  0]
 [ 0  0  0]
 [ 0 -1 -2]
 [-2 -1 -2]]
Action taken: [2, 0, 3, 1]
Reward: 0

Current player: 1
Board state
[[ 2  1  2]
 [ 0  1  2]
 [ 0  0  0]
 [-2  2  0]
 [ 0  0  0]
 [ 0 -1 -2]
 [-2 -1 -2]]
Action taken: [5, 1, 4, 1]
Reward: 0

Current player: 2
Board state
[[ 2  1  2]
 [ 0  1  2]
 [ 0  0  0]
 [-2  2  0]
 [ 0 -1  0]
 [ 0  0 -2]
 [-2 -1 -2]]
Action taken: [1, 1, 2, 1]
Reward: 0

Current player: 1
Board state
[[ 2  1  2]
 [ 0  0  2]
 [ 0  1  0]
 [-2  2  0]
 [ 

#### Build an Interactive UI with Pygame